In [1]:
import pandas as pd
from datetime import datetime

# Excel dosyasını yükleme
df = pd.read_excel('Bahcelievler DWDM re-use.xlsx')

# Tarih sütununu datetime formatına çevirme
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Port numaralarına göre verileri sıralama (Shelf/Slot sütununa göre)
df = df.sort_values(by=['Shelf/Slot', 'Date'])

# Çalışma süresi sütununu baştan tanımlama
df['Çalışma Süresi (Gün)'] = None

# Shelf/Slot değerine göre gruplama ve hesaplama
for name, group in df.groupby('Shelf/Slot'):
    # Gruplama sonrası tarih sırasına göre sıralanmış grup
    group = group.sort_values(by='Date')
    
    if len(group) > 1:
        # İki veya daha fazla satır varsa, eski tarihli olan satıra çalışma süresi yazılır
        for i in range(len(group) - 1):
            current_row_idx = group.index[i]
            next_row_idx = group.index[i + 1]
            df.at[current_row_idx, 'Çalışma Süresi (Gün)'] = (df.at[next_row_idx, 'Date'] - df.at[current_row_idx, 'Date']).days
        # Son tarihli olan satır bugüne kadar olan çalışma süresi ile güncellenir
        df.at[next_row_idx, 'Çalışma Süresi (Gün)'] = (datetime.now() - df.at[next_row_idx, 'Date']).days
    else:
        # Sadece bir satır varsa, günümüz tarihine göre çalışma süresi hesaplanır
        df.at[group.index[0], 'Çalışma Süresi (Gün)'] = (datetime.now() - group['Date'].iloc[0]).days

# Sonuçları yeni bir Excel dosyasına kaydetme
df.to_excel('DWDM_analiz.xlsx', index=False)

# Tablonun ilk birkaç satırını görüntüleme
print(df.head())


   Region    City              Location Software Load Shelf/Slot Card Type  \
88  ADANA  MERSİN  033_BAHCELIEVLER_B01   18.30.33.11        1/1        EC   
8   ADANA  MERSİN  033_BAHCELIEVLER_B01   18.30.33.11        1/1        EC   
67  ADANA  MERSİN  033_BAHCELIEVLER_B01   18.30.33.11       1/10    11QPA4   
95  ADANA  MERSİN  033_BAHCELIEVLER_B04   18.30.33.14     1/10      11QPA4   
81  ADANA  MERSİN  033_BAHCELIEVLER_B01   18.30.33.11       1/11   11STAR1   

   Mnemonic  Unit Part Number     I     J    K       Date Çalışma Süresi (Gün)  
88       EC  8DG60349AAAC03     NaN   NaN  NaN 2010-08-23                 4229  
8        EC    8DG59241ABAA02  90.0  42.0  2.0 2022-03-22                  877  
67   11QPA4    8DG60349AAAC03  90.0  46.0  2.0 2022-03-22                  877  
95   11QPA4  8DG60349AAAC03     NaN   NaN  NaN 2017-05-04                 2660  
81  11STAR1    8DG59249AAAB01  90.0  54.0  0.0 2018-03-06                 1477  


In [9]:
import pandas as pd
import plotly.express as px

# DWDM_analiz.xlsx dosyasını yükleme
df = pd.read_excel('DWDM_analiz1.xlsx')

# Veriyi tarih ve Shelf/Slot numarası ile sıralama
df_sorted = df.sort_values(by=['Date', 'Shelf/Slot'])

# Grafik oluşturma
fig = px.scatter(
    df_sorted,
    x='Date',
    y='Shelf/Slot',
    size='Çalışma Süresi (Gün)',
    color='Shelf/Slot',
    hover_name='Shelf/Slot',
    hover_data={'Date': True, 'Çalışma Süresi (Gün)': True},
    labels={'Date': 'Tarih', 'Shelf/Slot': 'Shelf/Slot Numara', 'Çalışma Süresi (Gün)': 'Çalışma Süresi (Gün)'},
    title='Çalışma Sürelerinin Zaman İçindeki Dağılımı'
)

# Grafik başlığı ve eksen etiketlerini ekleme
fig.update_layout(
    xaxis_title='Tarih',
    yaxis_title='Shelf/Slot Numara',
    xaxis=dict(
        tickformat='%Y-%m-%d'
    )
)

# Gösterim
fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed